# Generate anchor boxes

> Methods to generate anchor boxes of different aspect ratios. 

In [ ]:
# | default_exp anchor

In [ ]:
# | export
import inspect

import math
import numpy as np
from fastcore.foundation import L, mask2idxs
from fastcore.utils import gt, is_listy
from numpy.typing import ArrayLike
from typing import Union
import json
from collections import defaultdict
import warnings

from pybx.ops import named_idx
from pybx.basics import get_bx, stack_bxs_inplace, BX_TYPE, BaseBx, Bx, bbx, get_bx
from pybx.utils import get_edges, validate_boxes, as_tuple, reassign_label
from pybx.excepts import NoGroundTruthBxs

To generate anchor boxes, we need three basic information:

- Input image size, `image_sz`: To position our anchor boxes within the maximum 
coordinates (`width`, `height`) of the image.
- Feature map size, `feature_sz`: Feature map is the size (`width`, `height`) 
of the output of a convolutional operation. A $10\times10$ feature map would mean 
$10\times10$ local receptive field locations can be traced back into the 
input image. These 100 receptive field locations ($10\times10=100$) in the input image 
would act as our initial anchor box candidates.

![](https://ars.els-cdn.com/content/image/1-s2.0-S0925231217314169-gr4.jpg)

- Aspect ratio of anchor boxes, `asp_ratio`: To generate anchor boxes with 
different `width` to `height` ratio (default `asp_ratio=1`).

In [ ]:
# | export
def bx(
    image_sz: (int, tuple),
    feature_sz: (int, tuple),
    asp_ratio: float = None,
    clip: bool = True,
    named: bool = True,
    anchor_sfx: str = "a",
    min_visibility: float = 0.25,
) -> ArrayLike:
    """Calculate anchor box coords given an image size and feature size
    for a single aspect ratio.

    Parameters
    ----------
    image_sz : (int,tuple)
        image size (width, height)
    feature_sz : (int,tuple)
        feature map size (width, height)
    asp_ratio : float, optional
        aspect ratio (width:height), by default None
    clip : bool, optional
        whether to apply np.clip, by default True
    named : bool, optional
        whether to return (coords, labels), by default True
    anchor_sfx : str, optional
        suffix anchor label with anchor_sfx, by default "a"
    min_visibility : float, optional
        minimum visibility dictates the condition for a box to be considered
        valid. The value corresponds to the ratio of expected area of an anchor box
        to the calculated area after clipping to image dimensions., by default 0.25

    Returns
    -------
    ArrayLike
        anchor box coordinates in `pascal_voc` format
        if named=True, a list of anchor box labels are also returned.
    """
    labels = None
    image_sz = as_tuple(image_sz)
    feature_sz = as_tuple(feature_sz)
    asp_ratio = 1.0 if asp_ratio is None else asp_ratio
    # n_boxes = __mul__(*feature_sz)
    top_edges = get_edges(image_sz, feature_sz, op="noop")
    bot_edge = get_edges(image_sz, feature_sz, op="add")
    coords = np.hstack([top_edges, bot_edge])  # raw coords
    coords_wh = coords[:, 2:] - coords[:, :2]  # w -> xmax-xmin, h -> ymax-ymin
    coords_center = coords[:, 2:] - coords_wh / 2  # xmax-w/2, ymax-h/2
    # scale the dimension of width and height with asp ratios
    _w = coords_wh[:, 0] * math.sqrt(asp_ratio)
    _h = coords_wh[:, 1] / math.sqrt(asp_ratio)
    coords_asp_wh = np.stack([_w, _h], -1)
    xy_min = coords_center - coords_asp_wh / 2
    xy_max = coords_center + coords_asp_wh / 2
    coords = np.hstack([xy_min, xy_max])
    # check for valid boxes
    b = validate_boxes(
        coords, image_sz, feature_sz, clip=clip, min_visibility=min_visibility
    )
    if named:
        anchor_sfx = f"{anchor_sfx}_{feature_sz[0]}x{feature_sz[1]}_{asp_ratio:.1f}_"
        labels = named_idx(len(b), anchor_sfx)
    # init multibx
    b = get_bx(
        b,
        labels,
    )
    return (b.coords, b.label) if named else b.coords

In [ ]:
coords_1, labels_1 = bx(
    100,
    10,
    0.5,
)

In [ ]:
coords_1

(#100) [[1, 0, 8, 12],[11, 0, 18, 12],[21, 0, 28, 12],[31, 0, 38, 12],[41, 0, 48, 12],[51, 0, 58, 12],[61, 0, 68, 12],[71, 0, 78, 12],[81, 0, 88, 12],[91, 0, 98, 12]...]

Usually multiple anchor boxes with different `feature_sz` and `asp_ratio` are 
needed. This requirement arises in the case of multiscale object detection.

For multiscale object detection, feature maps from different convolution 
operations of the network are used to trace back into the input image, to 
generate anchor boxes. 
The `bxs` method of `pybx` provides this possibility.

In [ ]:
# | export


def bxs(
    image_sz: (int, tuple),
    feature_szs: list = None,
    asp_ratios: list = None,
    named: bool = True,
    **kwargs,
) -> ArrayLike:
    """Calculate anchor box coords given an image size and multiple
    feature sizes for mutiple aspect ratios.

    Parameters
    ----------
    image_sz : (int,tuple)
        image size (width, height)
    feature_szs : list, optional
        list of feature map sizes, each feature map size being an int or tuple, by default [(8, 8), (2, 2)]
    asp_ratios : list, optional
        list of aspect ratios for anchor boxes, each aspect ratio being a float calculated by (width:height), by default [1 / 2.0, 1.0, 2.0]
    named : bool, optional
        whether to return (coords, labels), by default True

    Returns
    -------
    ArrayLike
        anchor box coordinates in pascal_voc format
        if named=True, a list of anchor box labels are also returned.
    """
    image_sz = as_tuple(image_sz)
    feature_szs = [8, 2] if feature_szs is None else feature_szs
    feature_szs = [as_tuple(fsz) for fsz in feature_szs]
    asp_ratios = [1 / 2.0, 1.0, 2.0] if asp_ratios is None else asp_ratios
    # always named=True for bx() call. named=True in fn signature of bxs() is in its scope.
    coords_ = [
        bx(image_sz, f, ar, named=True, **kwargs)
        for f in feature_szs
        for ar in asp_ratios
    ]
    coords_, labels_ = L(zip(*coords_))
    coords_ = np.vstack(coords_)
    labels_ = L([l_ for lab_ in labels_ for l_ in lab_])
    return (coords_, labels_) if named else np.vstack(coords_)

In [ ]:
coords, labels = bxs(100, [10, 8, 5, 2], [1, 0.5, 0.3])

In [ ]:
coords.shape, len(labels)

((587, 4), 587)

All methods work with asymetric `image_sz` (and or `feature_szs` as well):

In [ ]:
coords, labels = bxs((100, 200), [10, 8, 5, 2], [1, 0.5, 0.3])

In [ ]:
coords.shape, len(labels)

((336, 4), 336)

# Ground truth anchor boxes
Ground truth boxes are anchor boxes with maximum IOU with the true annotations.

Load actual annotations.

In [ ]:
true_annots = json.load(open("../data/annots.json"))
true_annots

[{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'},
 {'x_min': 13, 'y_min': 158, 'x_max': 90, 'y_max': 213, 'label': 'frame'}]

Convert to MultiBx for convenience:

In [ ]:
true_annots_as_bx = get_bx(true_annots)
true_annots_as_bx

MultiBx(coords: 2, labels: 2)

Generate anchor boxes for object detection task, given that we know: 

```py
image_sz = (256, 256)  # to know the upper bounds of candidate bounding boxes
feature_sz = [20, 10, 8, 3]
asp_ratio = [1, 0.5, 0.3]
```

In [ ]:
coords, labels = bxs((256, 256), [20, 10, 8, 3], [1, 0.5, 0.3])

In [ ]:
coords.shape, len(labels)

((1741, 4), 1741)

In [ ]:
n_boxes = coords.shape[0]
n_boxes

1741

Store coords as multibx for convenience.

In [ ]:
coords_as_bx = get_bx(coords=coords, label=labels)

Can use the true annotations and anchor boxes to calulate the IOU.

In [ ]:
true_annots_as_bx

MultiBx(coords: 2, labels: 2)

In [ ]:
len(true_annots_as_bx), len(coords_as_bx)

(2, 1741)

The question: for each true label in the provided true annotations, what are the possible ground truth anchor boxes?

In [ ]:
for annots in true_annots_as_bx:
    print(annots)

BaseBx(coords=[[130, 63, 225, 180]], label=['clock'])
BaseBx(coords=[[13, 158, 90, 213]], label=['frame'])


In [ ]:
gt_anchors_per_class = defaultdict(lambda: L())
iou_per_box = defaultdict(lambda: L())
for annots in true_annots_as_bx:
    label = annots.label[0]  # is a list of len 1
    ious = [annots.iou(coords_as_bx[i]) for i in range(n_boxes)]
    iou_per_box[label].extend(ious)

In [ ]:
iou_per_box.keys()

dict_keys(['clock', 'frame'])

In [ ]:
max(iou_per_box["clock"])

0.3482

In [ ]:
iou_per_box["clock"].argwhere(gt(0.3))  # indices of boxes with iou > 0.3

(#2) [1719,1728]

In [ ]:
iou_per_box["frame"].argwhere(gt(0.3))  # indices of boxes with iou > 0.3

(#3) [1720,1729,1738]

In [ ]:
(
    (max(iou_per_box["clock"])),
    max(iou_per_box["frame"]),
)  # add more anchor boxes to get better ground truth IOUs

(0.3482, 0.4488)

Improve the loop to return only those with good IOU:

In [ ]:
iou_thresh = 0.3
gt_anchors_per_class = defaultdict(lambda: L())
iou_per_box = defaultdict(lambda: L())
for annots in true_annots_as_bx:
    label = annots.label[0]  # is a list of len 1
    ious = L([annots.iou(coords_as_bx[i]) for i in range(n_boxes)])
    ious_filter = ious.argwhere(gt(iou_thresh))
    # report filtered box IOUs
    iou_per_box[label].extend(ious[ious_filter])
    # report selected boxes
    gt_anchors_per_class[label] = stack_bxs_inplace(
        *[coords_as_bx[i] for i in ious_filter]
    )

In [ ]:
iou_per_box["clock"], iou_per_box["frame"]

((#2) [0.34,0.3482], (#3) [0.3664,0.4488,0.3523])

In [ ]:
gt_anchors_per_class["clock"]

MultiBx(coords: 2, labels: 2)

Not very far from the original annotations. These scales and aspect ratios can also be read from the label. 
The finer the anchor boxes, the better the starting positions (ground truth anchor boxes).

In [ ]:
true_annots[0]

{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'}

In [ ]:
tmp = L([1, 2, 3])
msk = tmp.map(lambda x: x > 1)

In [ ]:
tmp[msk]

(#2) [2,3]

In [ ]:
mask2idxs(msk)

[1, 2]

In [ ]:
# | export


def get_gt_thresh_iou(
    true_annots,
    anchor_boxes,
    anchor_labels=None,
    iou_thresh=0.3,
    return_ious=False,
    return_masks=False,
    update_labels=True,
):
    """Calculate positive ground truth and extra positive ground truth bounding boxes based on iou threhsold.

    Can result in uneven number of positive ground truth boxes per class.

    Args:
        true_annots (Any): True annotations, typically in `pascal_voc` format
        anchor_boxes (Any): Candidate anchor boxes, typically calculated with `pybx.bxs`
        anchor_labels (List, optional): Anchor box labels, will be overwritten with true labels if `update_labels=True`. Defaults to None.
        iou_thresh (float, optional): IOU threshold to filter out negative ground truth anchor boxes. Defaults to 0.3.
        return_ious (bool, optional): Return IOU values for selected positive ground truth anchor boxes. Defaults to False.
        return_masks (bool, optional): Return boolean masks for all anchor boxes indicating if a box is positive (`True`) or negative (`False`) ground truth box. Defaults to False.
        update_labels (bool, optional): Overwrite with true annotations. Defaults to True.

    Returns:
        dict: positive ground truth anchor boxes per class
        dict: IOU of positive ground truth anchor boxes per class
        dict: boolean list indicating positive ground truth anchor boxes per class
    """
    gt_anchors_per_class = defaultdict(lambda: L())
    iou_per_class = defaultdict(lambda: L())
    mask_per_class = defaultdict(lambda: L())
    true_annots_as_bx = (
        get_bx(true_annots) if not isinstance(true_annots, BX_TYPE) else true_annots
    )
    anchor_labels = (
        [f"bx_{i}" for i in range(len(anchor_boxes))]
        if anchor_labels is None
        else anchor_labels
    )
    coords_as_bx = (
        get_bx(coords=anchor_boxes, label=anchor_labels)
        if not isinstance(anchor_boxes, BX_TYPE)
        else anchor_boxes
    )
    n_boxes = len(coords_as_bx)

    for annots in true_annots_as_bx:
        label = annots.label[0]  # is a list of len 1
        ious = L([round(annots.iou(coords_as_bx[i]), 4) for i in range(n_boxes)])
        # ious_filter = ious.argwhere(gt(iou_thresh))
        mask = ious.map(lambda x: x >= iou_thresh)
        ious_filter = mask2idxs(mask=mask)

        if mask.sum() < 1:
            warnings.warn(
                NoGroundTruthBxs(
                    f"No good ground truth anchors found for label={label}, try lowering threshold (iou_thresh={iou_thresh} or increasing candidates."
                )
            )
            gt_anchors_per_class[label] = None

        if return_ious:
            # report filtered box IOUs
            iou_per_class[label].extend(ious[ious_filter])
        if return_masks:
            mask_per_class[label].extend(mask)
        # report selected boxes
        # print([coords_as_bx[i] for i in ious_filter])
        if mask.sum() > 0:
            gt_anchors_per_class[label] = stack_bxs_inplace(
                *[
                    (
                        reassign_label(coords_as_bx[i], label=[label])
                        if update_labels
                        else coords_as_bx[i]
                    )
                    for i in ious_filter
                ]
            )

    return dict(gt_anchors_per_class), dict(iou_per_class), dict(mask_per_class)

In [ ]:
coords

array([[  0,   0,  12,  12],
       [ 12,   0,  25,  12],
       [ 25,   0,  38,  12],
       ...,
       [ 19, 135,  66, 256],
       [104, 135, 151, 256],
       [189, 135, 236, 256]])

In [ ]:
gt_anchors_per_class, ious_per_class, mask_per_class = get_gt_thresh_iou(
    true_annots, coords, iou_thresh=0.3, return_ious=True
)
gt_anchors_per_class

{'clock': MultiBx(coords: 2, labels: 2),
 'frame': MultiBx(coords: 3, labels: 3)}

In [ ]:
mask_per_class

{}

In [ ]:
true_annots

[{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'},
 {'x_min': 13, 'y_min': 158, 'x_max': 90, 'y_max': 213, 'label': 'frame'}]

In [ ]:
ious_per_class["clock"]

(#2) [0.34,0.3482]

In [ ]:
gt_anchors_per_class["clock"].coords

[[170, 85, 256, 170], [183, 67, 243, 188]]

If anchor box labels are passed, they can be preserved instead of overwriting with ground truth labels.

In [ ]:
get_gt_thresh_iou(
    [100, 150, 180, 300, "hat"],
    coords,
    iou_thresh=0.3,
    anchor_labels=labels,
    update_labels=False,
    return_ious=True,
)

({'hat': MultiBx(coords: 3, labels: 3)},
 {'hat': (#3) [0.453,0.4899,0.3921]},
 {})

True annots can also be a list containing the label as the last item.

In [ ]:
get_gt_thresh_iou([100, 150, 180, 300, "hat"], coords, iou_thresh=0.3, return_ious=True)

({'hat': MultiBx(coords: 3, labels: 3)},
 {'hat': (#3) [0.453,0.4899,0.3921]},
 {})

In [ ]:
get_gt_thresh_iou(
    [[100, 150, 180, 300, "hat"], [100, 120, 280, 200, "shirt"]],
    coords,
    iou_thresh=0.3,
    return_ious=True,
)

/tmp/ipykernel_330543/1922175922.py:57: NoGroundTruthBxs: No good ground truth anchors found for label=shirt, try lowering threshold (iou_thresh=0.3 or increasing candidates.
  warnings.warn(


({'hat': MultiBx(coords: 3, labels: 3), 'shirt': None},
 {'hat': (#3) [0.453,0.4899,0.3921], 'shirt': (#0) []},
 {})

Method to also return just the max IOU ground truth boxes.

In [ ]:
# | export


def get_gt_max_iou(
    true_annots,
    anchor_boxes,
    anchor_labels=None,
    return_ious=False,
    return_masks=False,
    positive_boxes=1,
    update_labels=True,
):
    """Calculate positive ground truth and extra positive ground truth bounding boxes based on maximum IOU condition.

    Will always provide a box, therfore constant number `positive_boxes` of positive ground truth boxes per class.

    Args:
        true_annots (Any): True annotations, typically in `pascal_voc` format
        anchor_boxes (Any): Candidate anchor boxes, typically calculated with `pybx.bxs`
        anchor_labels (List, optional): Anchor box labels, will be overwritten with true labels if `update_labels=True`. Defaults to None.
        return_ious (bool, optional): Return IOU values for selected positive ground truth anchor boxes. Defaults to False.
        return_masks (bool, optional): Return boolean masks for all anchor boxes indicating if a box is positive (`True`) or negative (`False`) ground truth box. Defaults to False.
        update_labels (bool, optional): Overwrite with true annotations. Defaults to True.
        positive_boxes (int, optional): Number of extra/positive ground truth boxes to return. Defaults to 1.

    Returns:
        dict: positive ground truth anchor boxes per class
        dict: IOU of positive ground truth anchor boxes per class
        dict: boolean list indicating positive ground truth anchor boxes per class
    """
    gt_anchors_per_class = defaultdict(lambda: L())
    iou_per_class = defaultdict(lambda: L())
    mask_per_class = defaultdict(lambda: L())
    true_annots_as_bx = (
        get_bx(true_annots) if not isinstance(true_annots, BX_TYPE) else true_annots
    )
    anchor_labels = (
        [f"bx_{i}" for i in range(len(anchor_boxes))]
        if anchor_labels is None
        else anchor_labels
    )
    coords_as_bx = (
        get_bx(coords=anchor_boxes, label=anchor_labels)
        if not isinstance(anchor_boxes, BX_TYPE)
        else anchor_boxes
    )
    n_boxes = len(coords_as_bx)

    for annots in true_annots_as_bx:
        label = annots.label[0]  # is a list of len 1
        ious = L([round(annots.iou(coords_as_bx[i]), 4) for i in range(n_boxes)])
        ious_sorted = ious.sorted(reverse=True)
        max_iou = ious_sorted[:positive_boxes]
        ious_filter = [ious.index(m) for m in max_iou]
        mask = L([True if idx in ious_filter else False for idx in range(n_boxes)])

        if mask.sum() < 1:
            warnings.warn(
                NoGroundTruthBxs(
                    f"No good ground truth anchors found for label={label}, try increasing candidates."
                )
            )
            gt_anchors_per_class[label] = None

        # print(max_iou, ious_filter)
        if return_ious:
            # report filtered box IOUs
            iou_per_class[label].extend(ious[ious_filter])
        if return_masks:
            mask_per_class[label].extend(mask)
        # report selected boxes
        # print([coords_as_bx[i] for i in ious_filter])
        if mask.sum() > 0:
            gt_anchors_per_class[label] = stack_bxs_inplace(
                *[
                    (
                        reassign_label(coords_as_bx[i], label=[label])
                        if update_labels
                        else coords_as_bx[i]
                    )
                    for i in ious_filter
                ]
            )

    return dict(gt_anchors_per_class), dict(iou_per_class), dict(mask_per_class)

In [ ]:
get_gt_max_iou(true_annots, coords)

({'clock': MultiBx(coords: 1, labels: 1),
  'frame': MultiBx(coords: 1, labels: 1)},
 {},
 {})

Can also use methods for the box to calculate properties or convert to different box formats.

In [ ]:
tmp_bx = get_gt_max_iou(true_annots, coords)[0]["clock"]
tmp_bx[0]

BaseBx(coords=[[183, 67, 243, 188]], label=['clock'])

In [ ]:
tmp_bx[0].xywh

[[183, 67, 60, 121, 'clock']]

In [ ]:
tmp_bx[0].yolo(w=300, h=300, normalize=False)  # cx, cy, bw, bh

[[0.71, 0.425, 0.2, 0.4033, 'clock']]

In [ ]:
# here w h is the image w and h
tmp_bx[0].yolo(w=300, h=300, normalize=True)  # cx/w, cy/h, bw/w, bh/h

[[0.71, 0.425, 0.2, 0.4033, 'clock']]

In [ ]:
np.round([0.4046875, 0.840625, 0.503125, 0.24375], 4)

array([0.4047, 0.8406, 0.5031, 0.2438])

In [ ]:
true_annots

[{'x_min': 130, 'y_min': 63, 'x_max': 225, 'y_max': 180, 'label': 'clock'},
 {'x_min': 13, 'y_min': 158, 'x_max': 90, 'y_max': 213, 'label': 'frame'}]

In [ ]:
gt_anchors_per_class, ious_per_class, mask_per_class = get_gt_max_iou(
    true_annots,
    coords,
    return_ious=True,
    return_masks=True,
    positive_boxes=1,  # number of positive bounding boxes to allow
)
gt_anchors_per_class, ious_per_class, mask_per_class

({'clock': MultiBx(coords: 1, labels: 1),
  'frame': MultiBx(coords: 1, labels: 1)},
 {'clock': (#1) [0.3482], 'frame': (#1) [0.4488]},
 {'clock': (#1741) [False,False,False,False,False,False,False,False,False,False...],
  'frame': (#1741) [False,False,False,False,False,False,False,False,False,False...]})

Vocabulary:
- Ground truth bounding box
- Ground truth anchor box or positive anchor box
- Negative anchor box
- Offset is calculated as ground truth bounding box minus positive anchor box coordinates (not for all anchor boxes, use mask)
- Normalized offsets

# Calculate offsets for ground truth anchor boxes

`BaseBx` also supports calculation of bounding box offset by calling the `get_offset()` method.

In [ ]:
mask_per_class

{'clock': (#1741) [False,False,False,False,False,False,False,False,False,False...],
 'frame': (#1741) [False,False,False,False,False,False,False,False,False,False...]}

In [ ]:
gt_anchors_per_class

{'clock': MultiBx(coords: 1, labels: 1),
 'frame': MultiBx(coords: 1, labels: 1)}

In [ ]:
true_annots_bx = get_bx(true_annots)
true_annots_bx

MultiBx(coords: 2, labels: 2)

In [ ]:
gt_anchors_per_class["clock"]

MultiBx(coords: 1, labels: 1)

In [ ]:
gt_anchors_per_class["clock"]  # this is still a MultiBx

MultiBx(coords: 1, labels: 1)

In [ ]:
len(gt_anchors_per_class["clock"])

1

In [ ]:
gt_anchors_per_class["clock"][0]  # this is a BaseBx, which wont raise a warning

BaseBx(coords=[[183, 67, 243, 188]], label=['clock'])

In [ ]:
true_annots_bx[0].get_offset(gt_anchors_per_class["clock"])

/run/media/data1/projects/pybx/pybx/basics.py:686: BxViolation: Other should be BaseBx, got MultiBx
  warnings.warn(BxViolation(f"Other should be BaseBx, got MultiBx"))


(#4) [-5.9167,-0.4959,2.2977,-0.1681]

In [ ]:
true_annots_bx[0].get_offset(
    gt_anchors_per_class["clock"][0]
)  # by default normalize=True

(#4) [-5.9167,-0.4959,2.2977,-0.1681]

With `normalize=False`, it calculates simple difference between centers `(dcx, dcy)` and ratio of width and heights `log(w'/w), log(h'/h)`  the two boxes.

In [ ]:
true_annots_bx[0].get_offset(gt_anchors_per_class["clock"][0], normalize=False)  #

(#4) [-35.5,-6.0,95.0,117.0]

The following helper function repeats the same operation for multiple boxes, provided the masks (so that only ground truch anchor box offsets are calculated)

In [ ]:
# | export


def get_gt_offsets(
    true_annots: BaseBx,
    anchor_boxes,
    anchor_labels=None,  # do we need to pass this
    masks=None,
    sigma=(0.1, 0.2),
    normalize=True,
    log_func=np.log,
    update_labels=False,
):
    """
    Calculates the offset of the true annotations from the anchor boxes using the `get_offset` method of `BaseBx`.

    Args:
        true_annots (Any): True annotation for a single object, typically in `pascal_voc` format
        anchor_boxes (Any): Candidate anchor boxes, typically calculated with `pybx.bxs`
        anchor_labels (List, optional): Anchor box labels, will be overwritten with ground truth labels if `update_labels=True`. Defaults to None.
        masks (List, optional): Anchor box masks indicating if a box is positive/negative anchor box. If nothing is passed, offsets are calculated for all anchor boxes passed. Defaults to None.
        sigma (tuple, optional): Estimated of standard deviation for the distances and ratios. Defaults to (0.1, 0.2).
        normalize (bool, optional): Whether to normalize the offsets using the methods used in the SSD paper. Defaults to True.
        log_func (func, optional): Function for normalizing the ratio of widths and heights. Defaults to np.log.
        update_labels (bool, optional): Overwrite positive anchor boxes with object class and negative anchor boxes with `background` class. Defaults to False.

    Returns:
        list: List of all anchor box offsets.
        list: List of corresponding anchor box labels.
    """
    if not isinstance(true_annots, Bx):
        # the annotation for a single object, make listy if not
        true_annots = true_annots if is_listy(true_annots) else [true_annots]
        true_annots = get_bx(true_annots)[0]  # get back the BaseBx

    n_anchors = len(anchor_boxes)
    # no of anchor boxes (includes positive and negative anchor boxes)
    masks = (
        masks if masks is not None else L([True] * n_anchors)
    )  # if no masks provided, repeat for all anchors
    offsets = np.zeros((n_anchors, 4))
    labels = L(["background"] * n_anchors)  # if update_labels else anchor_labels
    true_label = true_annots.label[0] if len(true_annots.label) != 0 else "unknown"
    for idx, (box, mask) in enumerate(zip(anchor_boxes, masks)):
        if mask:
            offsets[idx, :] = true_annots.get_offset(
                box, normalize=normalize, sigma=sigma, log_func=log_func
            )
            # labels with mask=True will be the candidates for actual ground truth class.
            if update_labels:  # and (labels is not None):
                labels[idx] = true_label
    return offsets, labels

The Ground truth bounding boxes should ideally be passed in as a `BaseBx`, or will attempt a conversion to `BaseBx`.

In [ ]:
true_annots_bx[0], get_bx([true_annots[0]])[0]

(BaseBx(coords=[[130, 63, 225, 180]], label=['clock']),
 BaseBx(coords=[[130, 63, 225, 180]], label=['clock']))

In [ ]:
# passing dict
gt_offsets_clock, labels = get_gt_offsets(
    true_annots[0], coords, masks=mask_per_class["clock"]
)
gt_offsets_clock[
    mask_per_class["clock"]
]  # using the mask to look at only the valid offset

array([[-5.9167, -0.4959,  2.2977, -0.1681]])

In [ ]:
# passing list
gt_offsets_clock, labels = get_gt_offsets(
    [130, 63, 225, 180, "clock"], coords, masks=mask_per_class["clock"]
)
gt_offsets_clock[mask_per_class["clock"]]

array([[-5.9167, -0.4959,  2.2977, -0.1681]])

Updating the labels of the candidates to actual ground truth class labels might be a nice addition as well.

In [ ]:
gt_offsets_clock, labels = get_gt_offsets(
    true_annots[0], coords, masks=mask_per_class["clock"], update_labels=False
)
labels.unique()

(#1) ['background']

In [ ]:
gt_offsets_clock, labels = get_gt_offsets(
    true_annots[0], coords, masks=mask_per_class["clock"], update_labels=True
)
labels.unique()

(#2) ['background','clock']

If no ground truth label is passed with `update_labels=True`, then `unknown` is assigned. Doesn't work with `dict`s.

In [ ]:
gt_offsets_clock, labels = get_gt_offsets(
    [130, 63, 225, 180], coords, masks=mask_per_class["clock"], update_labels=True
)
labels.unique()

(#2) ['background','unknown']

In [ ]:
gt_offsets_clock[:, 0]

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
gt_offsets_clock[
    mask_per_class["clock"]
]  # czan use the mask to index the only valid offset

array([[-5.9167, -0.4959,  2.2977, -0.1681]])

The log of widths and heights can be skipped or modified if a different `log_func` is passed `log(w'/w), log(h'/h)`.

In [ ]:
from fastcore.foundation import (
    noop,
)  # do no operation, basically an identity function f(x) = x

In [ ]:
gt_offsets_clock_nolog, _ = get_gt_offsets(
    true_annots_bx[0], coords, masks=mask_per_class["clock"], log_func=noop
)
gt_offsets_clock_nolog[mask_per_class["clock"]]

array([[-5.9167, -0.4959,  7.9167,  4.8347]])

Or a custom funciton.

In [ ]:
gt_offsets_clock_nolog, _ = get_gt_offsets(
    true_annots_bx[0],
    coords,
    masks=mask_per_class["clock"],
    log_func=lambda x: x * np.log2(x),
)
gt_offsets_clock_nolog[mask_per_class["clock"]]

array([[-5.9167, -0.4959,  5.2485, -0.2345]])

Repeating the same operations for all classes.

In [ ]:
for idx, true_bx in enumerate(true_annots_bx):
    get_gt_offsets(true_bx, coords, masks=mask_per_class[true_bx.label[0]])

In [ ]:
true_annots_bx

MultiBx(coords: 2, labels: 2)

If a mask is not provided, offsets are calculated for all anchors, which makes the process slow.

In [ ]:
%%timeit
get_gt_offsets(true_annots_bx[0], coords)  # without mask

135 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
%%timeit
get_gt_offsets(true_annots_bx[0], coords, masks=mask_per_class["clock"])  # with mask

468 µs ± 90.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
# |hide
from nbdev import nbdev_export

nbdev_export()